# Install Dependencies
---

In [ ]:
# for installing packages using legacy code
!pip install setuptools

# The environment we will be working with
!pip install git+https://github.com/ParisaAligol/CityLearn-EVModel-Parisa@develop-ev

# For participant interactions (buttons)
!pip install ipywidgets

# To generate static figures
!pip install matplotlib
!pip install seaborn

# Provide standard RL algorithms
!pip install stable_baselines3
! pip install shimmy


  Cloning https://github.com/ParisaAligol/CityLearn-EVModel-Parisa (to revision develop-ev) to /tmp/pip-req-build-zy8fat0z
  Running command git clone --filter=blob:none --quiet https://github.com/ParisaAligol/CityLearn-EVModel-Parisa /tmp/pip-req-build-zy8fat0z
  Running command git checkout -b develop-ev --track origin/develop-ev
  Switched to a new branch 'develop-ev'
  Branch 'develop-ev' set up to track remote branch 'develop-ev' from 'origin'.
  Resolved https://github.com/ParisaAligol/CityLearn-EVModel-Parisa to commit 1562f1890ebfdf9526f60566e21bc7858cfce0ca
  Preparing metadata (setup.py) ... done


# Import Dependencies
---

In [ ]:
import os
import numpy as np
import pandas as pd
import shutil
from stable_baselines3 import SAC
from citylearn.agents.rbc import BasicRBC
from citylearn.callback import SaveDataCallback
from citylearn.citylearn import CityLearnEnv
from citylearn.data import DataSet
from citylearn.energy_model import ElectricVehicle, ZERO_DIVISION_CAPACITY
from citylearn.reward_function import RewardFunction
from citylearn.utilities import read_json, write_json
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper

/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [ ]:
from google.colab import files

# Use the files.upload() function to upload a file
uploaded = files.upload()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Saving schema.json to schema (2).json
Saving weather.csv to weather (2).csv
Saving Building_1_EV_5.csv to Building_1_EV_5 (2).csv
Saving Building_2.csv to Building_2 (2).csv
Saving Building_2_EV_1.csv to Building_2_EV_1 (2).csv
Saving Building_2_EV_2.csv to Building_2_EV_2 (2).csv
Saving Building_2_EV_3.csv to Building_2_EV_3 (2).csv
Saving Building_3.csv to Building_3 (2).csv
Saving Building_4.csv to Building_4 (2).csv
Saving Building_4_EV_1.csv to Building_4_EV_1 (2).csv
Saving Building_4_EV_2.csv to Building_4_EV_2 (2).csv
Saving Building_4_EV_3.csv to Building_4_EV_3 (2).csv
Saving Building_4_EV_4.csv to Building_4_EV_4 (2).csv
Saving Building_4_EV_5.csv to Building_4_EV_5 (2).csv
Saving Building_5.csv to Building_5 (2).csv
Saving Building_6.csv to Building_6 (2).csv
Saving Building_6_EV_1.csv to Building_6_EV_1 (2).csv
Saving Building_6_EV_2.csv to Building_6_EV_2 (2).csv
Saving Building_6_EV_3.csv to Building_6_EV_3 (2).csv
Saving Building_6_EV_4.csv to Building_6_EV_4 (2).csv
Sa

In [ ]:
# Specify the destination directory path in the Colab environment
destination_directory = '/usr/local/lib/python3.10/dist-packages/citylearn/data/citylearn_challenge_2023_phase_1/'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

# Move the uploaded file to the destination directory
for filename, content in uploaded.items():
    file_path = os.path.join(destination_directory, filename)
    with open(file_path, 'wb') as f:
        f.write(content)

print("File moved to the specific directory successfully.")

File moved to the specific directory successfully.


In [ ]:
SCHEMA_TEMPLATE = 'citylearn_challenge_2023_phase_1'
TARGET_DATASET_DIRECTORY = 'ev_test_data'
SCHEMA_FILEPATH = os.path.join(TARGET_DATASET_DIRECTORY, SCHEMA_TEMPLATE, 'schema (1).json')

# Create Test Dataset
---

In [ ]:
# Copy an existing dataset
if os.path.isdir(TARGET_DATASET_DIRECTORY):
    shutil.rmtree(TARGET_DATASET_DIRECTORY)
else:
    pass

DataSet.copy(SCHEMA_TEMPLATE, TARGET_DATASET_DIRECTORY)
schema = read_json(SCHEMA_FILEPATH)



# Test Simulation
---

## RBC

In [ ]:
env = CityLearnEnv(SCHEMA_FILEPATH)
model = BasicRBC(env)
model.learn(episodes=1)
display(env.evaluate())

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


,cost_function,value,name,level
0,annual_peak_average,0.961009,District,district
1,carbon_emissions_total,1.008696,District,district
2,cost_total,0.992876,District,district
3,daily_peak_average,1.134671,District,district
4,discomfort_delta_average,0.000000,District,district
...,...,...,...,...
79,discomfort_too_cold_proportion,NaN,Building_7,building
80,discomfort_too_hot_proportion,NaN,Building_7,building
81,discomfort_delta_minimum,0.000000,Building_7,building
82,discomfort_delta_maximum,0.000000,Building_7,building


## SB3 SAC

In [ ]:
class CustomReward(RewardFunction):
    def __init__(self, env: CityLearnEnv):
        super().__init__(env)

    def calculate(self):
        reward_list = []

        for b in self.env.buildings:
            e = b.net_electricity_consumption[-1]
            reward = 0.0

            for s in [b.electrical_storage] + b.electric_vehicles:
                if not isinstance(s, ElectricVehicle) or (isinstance(s, ElectricVehicle) and s.schedule.availability[-1] == 1):
                    c = s.capacity_history[0]
                    s = s.soc[-1]/c
                    reward += -(1.0 + np.sign(e)*s)*abs(e) if c > ZERO_DIVISION_CAPACITY else 0.0
                else:
                    pass

            reward_list.append(reward)

        if self.env.central_agent:
            reward = [sum(reward_list)]
        else:
            reward = reward_list

        return reward

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# train
episodes = 10
env = CityLearnEnv(SCHEMA_FILEPATH, central_agent=True)
env.reward_function = CustomReward(env) # set custom reward
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
model = SAC('MlpPolicy', env, learning_starts=env.unwrapped.time_steps)
callback = SaveDataCallback(env, 'citylearn_ev', 'results')
model.learn(total_timesteps=episodes*env.unwrapped.time_steps, callback=callback)

# evaluate
observations = env.reset()
done = False

while not env.done:
    print(f'\rTime step: {env.time_step + 1}/{env.time_steps}', end=' '*50)
    actions, _ = model.predict(observations, deterministic=True)
    observations, reward, done, _ = env.step(actions)

display(env.evaluate())

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, ax

Time step: 8759/8760                                                  

,cost_function,value,name,level
0,annual_peak_average,1.024833,District,district
1,carbon_emissions_total,1.032672,District,district
2,cost_total,1.029403,District,district
3,daily_peak_average,1.096223,District,district
4,discomfort_delta_average,0.000000,District,district
...,...,...,...,...
79,discomfort_too_cold_proportion,NaN,Building_7,building
80,discomfort_too_hot_proportion,NaN,Building_7,building
81,discomfort_delta_minimum,0.000000,Building_7,building
82,discomfort_delta_maximum,0.000000,Building_7,building
